# Redudancy Analysis (RDA) 
### Genetic variation ~ geography + population structure

<hr style='border:2px solid gray'>

##### Adam Koller
 August 2024

### Loading R magic

In [1]:
%load_ext rpy2.ipython

### Loading libraries

In [2]:
%%R
library(vegan)
library(dplyr)

R[write to console]: Loading required package: permute

R[write to console]: Loading required package: lattice

R[write to console]: This is vegan 2.6-6.1

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Loading individual genotype matrix

In [15]:
%%R
snp.df = read.table('../data/genotypes/genotype_matrix.csv', sep=',', row.names=1, header=TRUE)

### Getting coordinate data

In [14]:
%%R
bams.df = read.table('../data/merged_bams/bams_4_rcr.txt', col.names=c('Sample_ID'))
bams.df$Sample_ID = sub("\\.bam$","", basename(bams.df$Sample_ID))
meta.df = read.csv('../config/meta_apriori_jittered.csv', row.names=1, header=TRUE)
coords.df = merge(bams.df, meta.df, by = "Sample_ID") %>% select(c('longitude','latitude'))

#### Transforming coords to dbMEM vectors

In [25]:
%%R
coords.dmatrix = dist(coords.df)
pcnm.out = pcnm(coords.dmatrix)
pcnm.vectors = as.data.frame(scores(pcnm.out))

### Loading ancestry coefficient data from STRUCTURE (*K* = 3)

In [21]:
%%R
q.df = read.csv('../data/structure/q_matrix_3', row.names=1)[,-1]

### Null and full baseline models for pcnm selection

In [26]:
%%R
RDA.pcnm.null = rda(snp.df ~ 1, pcnm.vectors)
RDA.pcnm.full = rda(snp.df ~ ., pcnm.vectors)

### Foward selection to select significant pcnm vectors

In [29]:
%%R
fwd.sel <- ordiR2step(RDA.pcnm.null, 
                      scope = formula(RDA.pcnm.full),
                     direction = "forward",
                     R2permutations = 1000,
                     R2scope = TRUE,
                     pstep=1000,
                     trace = 0)

In [31]:
%%R 
fwd.sel$call

Call: rda(formula = snp.df ~ PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 +
PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 +
PCNM34 + PCNM29 + PCNM15 + PCNM69 + PCNM20, data = pcnm.vectors)

               Inertia Proportion Rank
Total         310.5827     1.0000     
Constrained    40.9164     0.1317   18
Unconstrained 269.6663     0.8683  225
Inertia is variance 

Eigenvalues for constrained axes:
 RDA1  RDA2  RDA3  RDA4  RDA5  RDA6  RDA7  RDA8  RDA9 RDA10 RDA11 RDA12 RDA13 
8.180 6.667 2.617 2.453 2.293 2.025 1.838 1.749 1.584 1.488 1.446 1.443 1.301 
RDA14 RDA15 RDA16 RDA17 RDA18 
1.283 1.263 1.154 1.086 1.047 

Eigenvalues for unconstrained axes:
   PC1    PC2    PC3    PC4    PC5    PC6    PC7    PC8 
3.0494 2.8760 2.7390 2.6769 2.6759 2.6410 2.5646 2.5361 
(Showing 8 of 225 unconstrained eigenvalues)



### Model with all significant PCNM vectors and q-values

In [32]:
%%R
all_variables = cbind(q.df, pcnm.vectors)

rda.full = rda(snp.df ~ PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 +
                        PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 +
                        PCNM34 + PCNM29 + PCNM15 + PCNM69 + PCNM20 + prop_1 + prop_2 + prop_3,
                        data= all_variables)

In [34]:
%%R
RsquareAdj(rda.full)

$r.squared
[1] 0.148951

$adj.r.squared
[1] 0.06844631



PCNM vectors and ancestry coefficients explain 14.9% (adjusted to 6.8% for the number of explanatory variables) of the variation in individual SNP data

##### Significance of each term

In [35]:
%%R
anova.cca(rda.full, step = 1000, by='term')

Permutation test for rda under reduced model
Terms added sequentially (first to last)
Permutation: free
Number of permutations: 999

Model: rda(formula = snp.df ~ PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 + PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 + PCNM34 + PCNM29 + PCNM15 + PCNM69 + PCNM20 + prop_1 + prop_2 + prop_3, data = all_variables)
          Df Variance      F Pr(>F)    
PCNM1      1    5.838 4.9032  0.001 ***
PCNM4      1    5.072 4.2598  0.001 ***
PCNM3      1    4.408 3.7023  0.001 ***
PCNM5      1    2.534 2.1283  0.001 ***
PCNM9      1    2.478 2.0813  0.001 ***
PCNM2      1    2.336 1.9618  0.001 ***
PCNM6      1    2.083 1.7494  0.001 ***
PCNM7      1    1.862 1.5643  0.001 ***
PCNM8      1    1.617 1.3578  0.001 ***
PCNM10     1    1.541 1.2939  0.002 ** 
PCNM12     1    1.489 1.2510  0.003 ** 
PCNM13     1    1.487 1.2487  0.003 ** 
PCNM19     1    1.417 1.1903  0.012 *  
PCNM34     1    1.376 1.1555  0.024 *  
PCNM29     1    1.360 1.1419  0.027 

### Partial RDA (Population structure when controlling for geography)

I also ommitted non significant variables from the full model (PCNM69, PCNM20, and prop_3)

In [37]:
%%R
p.RDA.struct = rda(snp.df ~ prop_1 + prop_2 + 
                            Condition(PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 +
                                      PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 +
                                      PCNM34 + PCNM29 + PCNM15), data = all_variables)

In [38]:
%%R
RsquareAdj(p.RDA.struct)

$r.squared
[1] 0.013236

$adj.r.squared
[1] 0.005951409



##### Significance of each term

In [39]:
%%R
anova.cca(p.RDA.struct, step = 1000, by='term')

Permutation test for rda under reduced model
Terms added sequentially (first to last)
Permutation: free
Number of permutations: 999

Model: rda(formula = snp.df ~ prop_1 + prop_2 + Condition(PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 + PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 + PCNM34 + PCNM29 + PCNM15), data = all_variables)
          Df Variance      F Pr(>F)    
prop_1     1    1.998 1.6761  0.001 ***
prop_2     1    2.113 1.7723  0.001 ***
Residual 225  268.219                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


##### Significance of entire model

In [41]:
%%R
anova.cca(p.RDA.struct, step = 1000)

Permutation test for rda under reduced model
Permutation: free
Number of permutations: 999

Model: rda(formula = snp.df ~ prop_1 + prop_2 + Condition(PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 + PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 + PCNM34 + PCNM29 + PCNM15), data = all_variables)
          Df Variance      F Pr(>F)    
Model      2    4.111 1.7242  0.001 ***
Residual 225  268.219                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### Partial RDA (Geography when controlling for population structure)

In [43]:
%%R
p.RDA.geo = rda(snp.df ~ PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 +
                                      PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 +
                                      PCNM34 + PCNM29 + PCNM15 + 
                            Condition(prop_1 + prop_2), data = all_variables)

In [44]:
%%R
RsquareAdj(p.RDA.geo)

$r.squared
[1] 0.08509175

$adj.r.squared
[1] 0.02387666



##### Significance of each term

In [45]:
%%R
anova.cca(p.RDA.geo, step = 1000, by='term')

Permutation test for rda under reduced model
Terms added sequentially (first to last)
Permutation: free
Number of permutations: 999

Model: rda(formula = snp.df ~ PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 + PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 + PCNM34 + PCNM29 + PCNM15 + Condition(prop_1 + prop_2), data = all_variables)
          Df Variance      F Pr(>F)    
PCNM1      1    2.295 1.9251  0.001 ***
PCNM4      1    1.974 1.6563  0.001 ***
PCNM3      1    1.570 1.3172  0.001 ***
PCNM5      1    1.755 1.4720  0.001 ***
PCNM9      1    2.138 1.7934  0.001 ***
PCNM2      1    2.009 1.6852  0.001 ***
PCNM6      1    1.837 1.5413  0.001 ***
PCNM7      1    1.631 1.3679  0.001 ***
PCNM8      1    1.462 1.2261  0.001 ***
PCNM10     1    1.444 1.2113  0.001 ***
PCNM12     1    1.402 1.1762  0.001 ***
PCNM13     1    1.480 1.2417  0.001 ***
PCNM19     1    1.379 1.1568  0.003 ** 
PCNM34     1    1.361 1.1417  0.007 ** 
PCNM29     1    1.338 1.1224  0.011 *  
PCNM15     1

##### Significance of entire model

In [46]:
%%R
anova.cca(p.RDA.geo, step = 1000)

Permutation test for rda under reduced model
Permutation: free
Number of permutations: 999

Model: rda(formula = snp.df ~ PCNM1 + PCNM4 + PCNM3 + PCNM5 + PCNM9 + PCNM2 + PCNM6 + PCNM7 + PCNM8 + PCNM10 + PCNM12 + PCNM13 + PCNM19 + PCNM34 + PCNM29 + PCNM15 + Condition(prop_1 + prop_2), data = all_variables)
          Df Variance      F Pr(>F)    
Model     16   26.428 1.3856  0.001 ***
Residual 225  268.219                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
